# Movie Plume: End-to-End Demo

This notebook demonstrates a minimal end-to-end flow for using a movie-backed plume field:

- Ingest or select a tiny Zarr dataset (`concentration[t,y,x]`, float32)
- Load it via `MoviePlumeField` (step policy: wrap or clamp)
- Assemble a component-based environment using the factory with `plume="movie"`
- Step a few iterations and optionally preview output (headless-safe)

> Dependencies: optional media libs (xarray, zarr, imageio) are used when available; otherwise, the demo will skip ingest and print guidance.


In [ ]:
from pathlib import Path
import json

# Resolve repo root heuristically for local runs
repo = Path.cwd()
if not (repo / "src").exists():
    if (Path.cwd().parent / "src").exists():
        repo = Path.cwd().parent

# Prefer a small repo-provided dataset; otherwise, synthesize via CLI when media deps exist
sample_zarr = repo / "src" / "backend" / "tests" / "data" / "video" / "smoke.zarr"
out_root = repo / "notebooks" / "_data"
out_root.mkdir(parents=True, exist_ok=True)
out_zarr = out_root / "movie_demo.zarr"
sample_zarr, out_zarr

In [ ]:
# Ingest tiny frames to Zarr when sample is absent; gracefully skip if media libs unavailable
dataset_root: Path
if sample_zarr.exists():
    dataset_root = sample_zarr
else:
    try:
        # Create tiny grayscale frames directory
        import numpy as np
        import imageio.v3 as iio  # type: ignore

        frames_dir = out_root / "frames"
        frames_dir.mkdir(parents=True, exist_ok=True)
        for t in range(4):
            img = np.full((32, 32), t * 60, dtype=np.uint8)
            iio.imwrite(frames_dir / f"{t:03d}.png", img)
        # Run ingest CLI programmatically
        from plume_nav_sim.cli import video_ingest

        argv = [
            "--input",
            str(frames_dir),
            "--output",
            str(out_zarr),
            "--fps",
            "10",
            "--pixel-to-grid",
            "1 1",
            "--origin",
            "0 0",
            "--normalize",
        ]
        rc = int(video_ingest.main(argv))
        if rc != 0 or not out_zarr.exists():
            print("Ingest skipped or failed; media libs may be unavailable.")
            dataset_root = out_zarr  # may not exist; next cell will guard
        else:
            dataset_root = out_zarr
    except Exception as e:
        print("Media ingest unavailable (using placeholder path):", e)
        dataset_root = out_zarr  # placeholder; next cell will handle absence
dataset_root

In [ ]:
# Load MoviePlumeField and inspect metadata (if dataset exists)
from plume_nav_sim.plume.movie_field import MovieConfig, MoviePlumeField

if dataset_root.exists():
    field = MoviePlumeField(MovieConfig(path=str(dataset_root), step_policy="wrap"))
    info = {
        "grid_size": (field.grid_size.width, field.grid_size.height),
        "num_frames": field.num_frames,
        "attrs": (
            field.attrs.model_dump()
            if hasattr(field.attrs, "model_dump")
            else dict(field.attrs)
        ),
    }
    info
else:
    print(
        "Dataset not available. Ensure xarray/zarr/imageio are installed or use the repo sample."
    )
    info = {"grid_size": None, "num_frames": 0, "attrs": {}}
    info

In [ ]:
# Assemble environment via factory and step a few iterations
from plume_nav_sim.envs.factory import create_component_environment

if dataset_root.exists():
    env = create_component_environment(
        plume="movie",
        movie_path=str(dataset_root),
        movie_step_policy="wrap",
        action_type="discrete",
        observation_type="concentration",
        reward_type="step_penalty",
        max_steps=20,
    )
    obs, info = env.reset(seed=123)
    steps = 0
    terminated = truncated = False
    while not (terminated or truncated) and steps < 10:
        # Advance one step; underlying movie field maps step count to frame
        obs, r, terminated, truncated, info = env.step(env.action_space.sample())
        steps += 1
    print("Completed steps:", steps, "terminated=", terminated, "truncated=", truncated)
else:
    print("Skipping env assembly: dataset not available.")

## Notes

- The factory automatically resolves grid size from the movie dataset and wires the field into the environment.
- `movie_step_policy="wrap"` cycles frames; `"clamp"` holds on the last frame.
- For large datasets, prefer consolidated Zarr metadata and chunking for faster startup.

Render to HTML for docs:
```bash
make nb-render
```
